In [1]:
def getHeatMap(newdf,month,count,numberDevice):
    
    #Estrai giorno e ora
    newdf['day']=newdf['hours'].str[:2]
    newdf['h']=newdf['hours'].str[-3:-1]
    #Crea nuovo daframe per heatmap
    f=pd.DataFrame(columns=['h','count'])
    f['day']=newdf['day']
    f['h']=newdf['h'].astype(int)
    f['count']=newdf['count'].astype(int)
    #Crea una pivot table
    piv = pd.pivot_table(f, values="count",index=["h"], columns=["day"], fill_value=0)
    '''
    # This dictionary defines the colormap
    cdict = {'red':  ((0.0, 0.8, 0.8),   # no red at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0)),  # set to 0.8 so its not too bright at 1

        'green': ((0.0, 0.0, 0.0),   # set to 0.8 so its not too bright at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.8, 0.8)),  # no green at 1

        'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0))   # no blue at 1
       }
    #Creazione heatmap con grana 1 secondo
    GnRd = colors.LinearSegmentedColormap('GnRd', cdict)
    '''
    ax = sns.heatmap(piv, mask=piv==0,square=True,linewidths=0.8,cmap="RdYlGn",vmin=1,vmax=numberDevice) 
    ax.set_title('Numero dispositivi '+'EB'+str(count)+" "+month)
    ax.set_facecolor(color='#AFC7C7')
    figure = ax.get_figure() 
    heatDir=os.path.join(os.path.abspath('..'),os.path.join("heatmap_num_dev\\","EB"+str(count)))
    if not os.path.exists(heatDir):
        os.makedirs(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.png')
    figure.savefig(heatmapPath, dpi=400)
    plt.clf()
#Funzione che ritorna dataframe in base al mese   
def switch_demo(argument):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    if(argument=='Jan'):
        return jandf
    if(argument=='Feb'):
        return febdf
    if(argument=='Mar'):
        return mardf
    if(argument=='Apr'):
        return aprdf
    if(argument=='May'):
        return maydf
    if(argument=='Jun'):
        return jundf
    
#Funzione che assegna le modifiche al dataframe in base al mese
def switch_df(argument,df):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    if(argument=='Jan'):
        jandf=df
    if(argument=='Feb'):
        febdf=df
    if(argument=='Mar'):
        mardf=df
    if(argument=='Apr'):
        aprdf=df
    if(argument=='May'):
        maydf=df
    if(argument=='Jun'):
        jundf=df

def process(chunck,filepath,count):
    global jandf
    global febdf
    global mardf
    global aprdf
    global maydf
    global jundf
    df=chunck.loc[:, ('timestamp', 'power')]
    # generiamo la data nel formato corretto a partire dal timestamp unix
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize("GMT").dt.tz_convert('Europe/Rome')
    # arrotondiamo da ms al secondo pi� vicino
    df['date'] = df['date'].dt.round('1s')
    #df['date'] = pd.to_datetime(df['date'],format="%m/%d/%Y")
    df = df[['date', 'power']]
    # crea una time series settando la data come index
    df = df.set_index('date')
    #Raggruppa per data ed effettua la media dei power
    df=df.groupby('date')[['power']].mean()
    #Aggiungi una colonna contenente  data e ora per heatmap
    #df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")+
    newdf=pd.DataFrame(columns=['hours'])
    df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    monthList=['Jan','Feb','Mar','Apr','May','Jun']
    
    #Esegui i vari filtraggi e crea un dataframe contenente i valori per ogni mese
    for month in monthList:
        newdf=df[df['hours'].str.contains(month)]
        filterdf=switch_demo(month)
        filterdf.loc[filterdf.hours.isin(newdf.hours), 'bool'] =filterdf['bool']+1
        filterdf.loc[(filterdf['bool']==1) & (filterdf.hours.isin(newdf.hours)) , 'count'] = filterdf['count']+1
        switch_df(month,filterdf)
 


    
    
    









In [2]:
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pandas as pd
DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019","\Dati puntuali EB2 Gennaio-Giugno 2019","\Dati puntuali EB3 Gennaio-Giugno 2019","\Dati puntuali EB4 Gennaio-Giugno 2019","\Dati puntuali EB5 Gennaio-Giugno 2019","\Dati puntuali EB6 Gennaio-Giugno 2019","\Dati puntuali EB7 Gennaio-Giugno 2019","\Dati puntuali EB8 Gennaio-Giugno 2019","\Dati puntuali EB9 Gennaio-Giugno 2019"]
#DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019"]
count=0
for dirpath in DIRPATH:
    #Creazione dataframe vuoti per ogni mese
    jandf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-01'),
        end=pd.Timestamp('2019-01') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    febdf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-02'),
        end=pd.Timestamp('2019-02') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    mardf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-03'),
        end=pd.Timestamp('2019-03') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    aprdf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-04'),
        end=pd.Timestamp('2019-04') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    maydf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-05'),
        end=pd.Timestamp('2019-05') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    jundf = pd.DataFrame({
    'date': pd.date_range(
        start=pd.Timestamp('2019-06'),
        end=pd.Timestamp('2019-06') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
        freq='H',  # <--- try '3h', '6h', '12h' if you want
        closed='left'
            )
    })
    jandf = jandf.set_index('date')
    febdf = febdf.set_index('date')
    mardf = mardf.set_index('date')
    aprdf = aprdf.set_index('date')
    maydf = maydf.set_index('date')
    jundf = jundf.set_index('date')
    jandf['hours']=jandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    febdf['hours']=febdf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    mardf['hours']=mardf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    aprdf['hours']=aprdf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    maydf['hours']=maydf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    jundf['hours']=jundf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    jandf['count']= febdf['count']= mardf['count']= aprdf['count']= maydf['count']= jundf['count']=0
    count=count+1
    print("Start "+str(count))
    dirp=os.path.join(os.path.abspath('..'),"data"+dirpath+"\\")
    filelist=[f for f in os.listdir(dirp)]
    #Numero di appliance per casa
    numberDevice=len(filelist)-1
    for filepath in filelist:
        if("Meter" in filepath):
            continue
        print(filepath)
        jandf['bool']= febdf['bool']= mardf['bool']= aprdf['bool']= maydf['bool']= jundf['bool']=0
        dataPath=os.path.join(dirp,filepath)
        for chunck in pd.read_csv(dataPath, sep=';',chunksize=500000):
            process(chunck,filepath,count)
    #pd.set_option('display.max_rows', 10000)
    #display(jundf)
    getHeatMap(jandf,'Jan',count,numberDevice)
    getHeatMap(febdf,'Feb',count,numberDevice)
    getHeatMap(mardf,'Mar',count,numberDevice)
    getHeatMap(aprdf,'Apr',count,numberDevice)
    getHeatMap(maydf,'May',count,numberDevice)
    getHeatMap(jundf,'Jun',count,numberDevice)
    print('FiNE')

Start 1
Dati puntuali Frigo EB1 2019.csv
Dati puntuali Lavastoviglie EB1 2019.csv
Dati puntuali Lavatrice EB1 2019.csv
FiNE
Start 2
FiNE
Start 3
Dati puntuali FrigoEB3 2019.csv
Dati puntuali Microonde EB3 2019.csv
FiNE
Start 4
Dati puntuali Lavatrice EB4 2019.csv
Dati puntuali TV EB4 2019.csv
FiNE
Start 5
Dati puntuali Lavastoviglie EB5 2019.csv
Dati puntuali Lavatrice EB5 2019.csv
FiNE
Start 6
Dati puntuali Frigo EB6 2019.csv
Dati puntuali Lavastoviglie EB6 2019.csv
Dati puntuali Lavatrice EB6 2019.csv
Dati puntuali Microonde EB6 2019.csv
FiNE
Start 7
Dati puntuali Frigo EB7 2019.csv
Dati puntuali Lavastoviglie EB7 2019.csv
Dati puntuali Lavatrice EB7 2019.csv
Dati puntuali Microonde EB7 2019.csv
FiNE
Start 8
Dati puntuali Lavatrice EB8 2019.csv
FiNE
Start 9
Dati puntuali Frigo EB9 2019.csv
Dati puntuali Lavastoviglie EB9 2019.csv
Dati puntuali Lavatrice EB9 2019.csv
Dati puntuali Microonde EB9 2019.csv
FiNE
